# 第十三章 編寫測試 (Writing Tests)

## 章節概述

測試是 Go 語言的一等公民，內建的 testing 套件提供了強大而簡潔的測試框架。本章將深入探討 Go 的測試生態系統，包括單元測試、基準測試、測試覆蓋率分析等核心概念。

### 本章重點內容
- **測試基礎**: Go 測試的基本概念和約定
- **測試結構**: 如何組織和編寫測試程式碼
- **測試數據**: 管理測試資料和夾具
- **表格測試**: Go 特色的測試模式
- **基準測試**: 效能測試和分析
- **測試覆蓋率**: 程式碼覆蓋率分析
- **測試工具**: go test 命令和相關工具

### Go 測試的核心原則
1. **簡單性**: 測試應該簡單明瞭
2. **可讀性**: 測試即文檔
3. **獨立性**: 測試之間不應相互依賴
4. **快速性**: 測試應該能快速執行

In [ ]:
// Go 測試框架綜合示範
package main

import (
    "fmt"
    "math"
    "sort"
    "strings"
    "testing"
    "time"
)

// 待測試的業務邏輯函式

// Calculator 結構 - 簡單的計算器
type Calculator struct {
    precision int
}

// NewCalculator 建立新的計算器
func NewCalculator(precision int) *Calculator {
    return &Calculator{precision: precision}
}

// Add 加法運算
func (c *Calculator) Add(a, b float64) float64 {
    result := a + b
    return c.roundToPrecision(result)
}

// Subtract 減法運算
func (c *Calculator) Subtract(a, b float64) float64 {
    result := a - b
    return c.roundToPrecision(result)
}

// Multiply 乘法運算
func (c *Calculator) Multiply(a, b float64) float64 {
    result := a * b
    return c.roundToPrecision(result)
}

// Divide 除法運算
func (c *Calculator) Divide(a, b float64) (float64, error) {
    if b == 0 {
        return 0, fmt.Errorf("division by zero")
    }
    result := a / b
    return c.roundToPrecision(result), nil
}

// roundToPrecision 四捨五入到指定精度
func (c *Calculator) roundToPrecision(value float64) float64 {
    multiplier := math.Pow(10, float64(c.precision))
    return math.Round(value*multiplier) / multiplier
}

// StringProcessor 字串處理器
type StringProcessor struct{}

// Reverse 反轉字串
func (sp *StringProcessor) Reverse(s string) string {
    runes := []rune(s)
    for i, j := 0, len(runes)-1; i < j; i, j = i+1, j-1 {
        runes[i], runes[j] = runes[j], runes[i]
    }
    return string(runes)
}

// IsPalindrome 檢查是否為回文
func (sp *StringProcessor) IsPalindrome(s string) bool {
    s = strings.ToLower(strings.ReplaceAll(s, " ", ""))
    return s == sp.Reverse(s)
}

// WordCount 計算單字數量
func (sp *StringProcessor) WordCount(s string) int {
    s = strings.TrimSpace(s)
    if s == "" {
        return 0
    }
    return len(strings.Fields(s))
}

// SortSlice 排序切片（用於基準測試）
func SortSlice(data []int) {
    sort.Ints(data)
}

// 示範測試的概念（在實際專案中，這些會在 _test.go 檔案中）
func main() {
    fmt.Println("=== Go 測試框架示範 ===")
    
    // 在實際應用中，以下代碼會在測試檔案中執行
    fmt.Println("\n注意: 以下展示測試的概念")
    fmt.Println("實際測試需要在 *_test.go 檔案中編寫\n")
    
    // 示範基本測試概念
    demonstrateBasicTesting()
    
    // 示範表格測試概念
    demonstrateTableTesting()
    
    // 示範基準測試概念
    demonstrateBenchmarking()
}

// 示範基本測試的概念
func demonstrateBasicTesting() {
    fmt.Println("1. 基本單元測試概念:")
    
    // 這展示了測試的基本結構
    calc := NewCalculator(2)
    
    // 測試加法
    result := calc.Add(2.5, 3.7)
    expected := 6.2
    
    if result != expected {
        fmt.Printf("  ❌ Add測試失敗: 期望 %v, 得到 %v\n", expected, result)
    } else {
        fmt.Printf("  ✅ Add測試通過: %v\n", result)
    }
    
    // 測試除法（包含錯誤處理）
    _, err := calc.Divide(10, 0)
    if err == nil {
        fmt.Println("  ❌ Divide測試失敗: 應該返回錯誤")
    } else {
        fmt.Printf("  ✅ Divide錯誤處理測試通過: %v\n", err)
    }
    
    // 字串處理測試
    sp := &StringProcessor{}
    
    reversed := sp.Reverse("hello")
    expectedReversed := "olleh"
    
    if reversed != expectedReversed {
        fmt.Printf("  ❌ Reverse測試失敗: 期望 %s, 得到 %s\n", expectedReversed, reversed)
    } else {
        fmt.Printf("  ✅ Reverse測試通過: %s\n", reversed)
    }
}

// 示範表格測試概念
func demonstrateTableTesting() {
    fmt.Println("\n2. 表格測試概念:")
    
    // 定義測試案例表格
    testCases := []struct {
        name     string
        input    string
        expected bool
    }{
        {"簡單回文", "aba", true},
        {"複雜回文", "A man a plan a canal Panama", true},
        {"非回文", "hello", false},
        {"空字串", "", true},
        {"單字符", "a", true},
        {"數字回文", "12321", true},
        {"數字非回文", "12345", false},
    }
    
    sp := &StringProcessor{}
    
    passedTests := 0
    totalTests := len(testCases)
    
    for _, tc := range testCases {
        result := sp.IsPalindrome(tc.input)
        
        if result == tc.expected {
            fmt.Printf("  ✅ %s: 通過\n", tc.name)
            passedTests++
        } else {
            fmt.Printf("  ❌ %s: 失敗 (輸入: %q, 期望: %v, 得到: %v)\n", 
                tc.name, tc.input, tc.expected, result)
        }
    }
    
    fmt.Printf("  測試結果: %d/%d 通過\n", passedTests, totalTests)
}

// 示範基準測試概念
func demonstrateBenchmarking() {
    fmt.Println("\n3. 基準測試概念:")
    
    // 模擬基準測試
    benchmarkCases := []struct {
        name string
        size int
    }{
        {"小資料集", 100},
        {"中資料集", 1000},
        {"大資料集", 10000},
    }
    
    for _, bc := range benchmarkCases {
        // 準備測試資料
        data := make([]int, bc.size)
        for i := range data {
            data[i] = bc.size - i // 逆序資料
        }
        
        // 測量執行時間
        start := time.Now()
        
        // 執行多次以獲得更準確的測量
        iterations := 1000
        if bc.size > 1000 {
            iterations = 100 // 大資料集減少迭代次數
        }
        
        for i := 0; i < iterations; i++ {
            testData := make([]int, len(data))
            copy(testData, data)
            SortSlice(testData)
        }
        
        elapsed := time.Since(start)
        avgTime := elapsed / time.Duration(iterations)
        
        fmt.Printf("  📊 %s (大小: %d): 平均時間 %v (%d 次迭代)\n", 
            bc.name, bc.size, avgTime, iterations)
    }
    
    fmt.Println("\n  基準測試提示:")
    fmt.Println("  - 使用 go test -bench=. 執行基準測試")
    fmt.Println("  - 使用 go test -benchmem 查看記憶體分配")
    fmt.Println("  - 使用 go test -cpuprofile 進行 CPU 分析")
}

/*
以下是實際測試檔案的範例結構 (calculator_test.go):

package main

import "testing"

func TestCalculatorAdd(t *testing.T) {
    calc := NewCalculator(2)
    result := calc.Add(2.5, 3.7)
    expected := 6.2
    
    if result != expected {
        t.Errorf("Add(2.5, 3.7) = %v; want %v", result, expected)
    }
}

func TestCalculatorDivideByZero(t *testing.T) {
    calc := NewCalculator(2)
    _, err := calc.Divide(10, 0)
    
    if err == nil {
        t.Error("Divide by zero should return an error")
    }
}

func TestIsPalindrome(t *testing.T) {
    tests := []struct {
        name     string
        input    string
        expected bool
    }{
        {"simple palindrome", "aba", true},
        {"complex palindrome", "A man a plan a canal Panama", true},
        {"not palindrome", "hello", false},
    }
    
    sp := &StringProcessor{}
    
    for _, tt := range tests {
        t.Run(tt.name, func(t *testing.T) {
            result := sp.IsPalindrome(tt.input)
            if result != tt.expected {
                t.Errorf("IsPalindrome(%q) = %v; want %v", tt.input, result, tt.expected)
            }
        })
    }
}

func BenchmarkSortSlice(b *testing.B) {
    data := make([]int, 1000)
    for i := range data {
        data[i] = 1000 - i
    }
    
    b.ResetTimer()
    
    for i := 0; i < b.N; i++ {
        testData := make([]int, len(data))
        copy(testData, data)
        SortSlice(testData)
    }
}

常用的測試命令:
- go test                    // 執行當前套件的測試
- go test ./...             // 執行所有套件的測試
- go test -v                // 詳細輸出
- go test -run TestAdd      // 只執行特定測試
- go test -bench=.          // 執行基準測試
- go test -cover            // 顯示測試覆蓋率
- go test -coverprofile=c.out // 生成覆蓋率報告
- go tool cover -html=c.out // 查看 HTML 覆蓋率報告
*/